# Baseline solution using library Natasha

## Load data

In [1]:
from zipfile import ZipFile
import pandas as pd

In [4]:
with ZipFile("../data/public_dat.zip", "r") as myzip:
    myzip.extractall(path='../data',  members=['dev.jsonl', 'test.jsonl', 'train.jsonl'])

In [36]:
test = pd.read_json('./data/test.jsonl', lines=True)

test.head()

,senences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588


In [4]:
# !pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3c0fd1e342b3de6e5b67e46ee7a129f42ad33704e15947d92a9abf27888726f2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=ef056f08c9d64920f755401bb3591f3d0e38ecaca522a285c81324803c310a03
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [13]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger,NamesExtractor, Doc

## Prediction for test data


In [44]:
out = []
for i in range(len(test)):
    d = {'id': int(test['id'][i]), 'ners': []}
    # using library Natasha
    sentence = Doc(test['senences'][i])
    segmenter = Segmenter()
    sentence.segment(segmenter)
    emb = NewsEmbedding()
    ner_tagger = NewsNERTagger(emb)
    sentence.tag_ner(ner_tagger)
    for item in sentence.ner.spans:
        tag = ''
        if item.type == 'ORG':
            tag = 'ORGANIZATION'
        elif item.type == 'PER':
            tag = 'PERSON'
        elif item.type == 'LOC':
            tag = 'LOCATION'
        d['ners'].append([int(item.start), int(item.stop) - 1, tag])
    out.append(d)

## Write to JSONL

In [49]:
import json

with open('test.jsonl', 'w') as f:
    for item in out:
        json.dump(item, f, default=str)
        f.write('\n')

!zip text test.jsonl

updating: test.jsonl (deflated 78%)
